In [1]:
import torch
from model.dit import DiT_models
from model.vae import VAE_models
from torchvision.io import read_video, write_video
from utils import one_hot_actions, sigmoid_beta_schedule
from tqdm import tqdm
from einops import rearrange
from torch import autocast

/home/iker/miniconda3/envs/oasis/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
assert torch.cuda.is_available()
device = "cuda:0"
ckpt = torch.load("checkpoints/oasis500m.pt")
model = DiT_models["DiT-S/2"]()
model.load_state_dict(ckpt, strict=False)
model = model.to(device).eval()

/tmp/ipykernel_35235/3437294327.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("checkpoints/oasis500m.pt")


In [3]:
model

DiT(
  (x_embedder): PatchEmbed(
    (proj): Conv2d(16, 1024, kernel_size=(2, 2), stride=(2, 2))
    (norm): Identity()
  )
  (t_embedder): TimestepEmbedder(
    (mlp): Sequential(
      (0): Linear(in_features=256, out_features=1024, bias=True)
      (1): SiLU()
      (2): Linear(in_features=1024, out_features=1024, bias=True)
    )
  )
  (spatial_rotary_emb): RotaryEmbedding()
  (temporal_rotary_emb): RotaryEmbedding()
  (external_cond): Linear(in_features=25, out_features=1024, bias=True)
  (blocks): ModuleList(
    (0-15): 16 x SpatioTemporalDiTBlock(
      (s_norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=False)
      (s_attn): SpatialAxialAttention(
        (to_qkv): Linear(in_features=1024, out_features=3072, bias=False)
        (to_out): Linear(in_features=1024, out_features=1024, bias=True)
        (rotary_emb): RotaryEmbedding()
      )
      (s_norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=False)
      (s_mlp): Mlp(
        (fc1): Linear(in_features=1024

In [4]:
vae_ckpt = torch.load("checkpoints/vit-l-20.pt")
vae = VAE_models["vit-l-20-shallow-encoder"]()
vae.load_state_dict(vae_ckpt)
vae = vae.to(device).eval()

/tmp/ipykernel_35235/631049858.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vae_ckpt = torch.load("checkpoints/vit-l-20.pt")


In [5]:
vae

AutoencoderKL(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1024, kernel_size=(20, 20), stride=(20, 20))
    (norm): Identity()
  )
  (encoder): ModuleList(
    (0-5): 6 x AttentionBlock(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=True)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (enc_norm): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
  (qu

In [6]:
B = 1  # Batch size
total_frames = 32  # Number of frames to generate
max_noise_level = 1000  # Maximum noise level for diffusion
ddim_noise_steps = 100  # Number of denoising steps
noise_range = torch.linspace(
    -1, max_noise_level - 1, ddim_noise_steps + 1
)  # Noise range
noise_abs_max = 20  # Maximum absolute value for noise
ctx_max_noise_idx = ddim_noise_steps // 10 * 3  # Context maximum noise index

In [7]:
noise_range

tensor([ -1.,   9.,  19.,  29.,  39.,  49.,  59.,  69.,  79.,  89.,  99., 109.,
        119., 129., 139., 149., 159., 169., 179., 189., 199., 209., 219., 229.,
        239., 249., 259., 269., 279., 289., 299., 309., 319., 329., 339., 349.,
        359., 369., 379., 389., 399., 409., 419., 429., 439., 449., 459., 469.,
        479., 489., 499., 509., 519., 529., 539., 549., 559., 569., 579., 589.,
        599., 609., 619., 629., 639., 649., 659., 669., 679., 689., 699., 709.,
        719., 729., 739., 749., 759., 769., 779., 789., 799., 809., 819., 829.,
        839., 849., 859., 869., 879., 889., 899., 909., 919., 929., 939., 949.,
        959., 969., 979., 989., 999.])

In [8]:
ctx_max_noise_idx

30

In [9]:
video_id = "snippy-chartreuse-mastiff-f79998db196d-20220401-224517.chunk_001"
mp4_path = f"sample_data/{video_id}.mp4"
actions_path = f"sample_data/{video_id}.actions.pt"
video = (
    read_video(mp4_path, pts_unit="sec")[0].float() / 255
)  # Normalize pixel values to [0, 1] range
actions = one_hot_actions(torch.load(actions_path))
offset = 100
video = video[offset : offset + total_frames].unsqueeze(
    0
)  # Selects a specific segment using offset
actions = actions[offset : offset + total_frames].unsqueeze(
    0
)  # Selects a specific segment using offset

/tmp/ipykernel_35235/354781882.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  actions = one_hot_actions(torch.load(actions_path))


In [10]:
video.size()

torch.Size([1, 32, 360, 640, 3])

In [11]:
actions.size()

torch.Size([1, 32, 25])

In [12]:
n_prompt_frames = 8
x = video[:, :n_prompt_frames]
x = x.to(device)
actions = actions.to(device)

x.size()

torch.Size([1, 8, 360, 640, 3])

In [13]:
scaling_factor = 0.07843137255
x = rearrange(x, "b t h w c -> (b t) c h w")
print(x.size())
H, W = x.shape[-2:]
with torch.no_grad():
    x = vae.encode(x * 2 - 1).mean * scaling_factor

x.size()

torch.Size([8, 3, 360, 640])


torch.Size([8, 576, 16])

In [14]:
x = rearrange(
    x,
    "(b t) (h w) c -> b t c h w",
    t=n_prompt_frames,
    h=H // vae.patch_size,
    w=W // vae.patch_size,
)

x.size()

torch.Size([1, 8, 16, 18, 32])

In [15]:
betas = sigmoid_beta_schedule(max_noise_level).to(device) 
print(betas.size())
alphas = 1.0 - betas
print(alphas.size())
alphas_cumprod = torch.cumprod(alphas, dim=0)
print(alphas_cumprod.size())
alphas_cumprod = rearrange(alphas_cumprod, "T -> T 1 1 1")
print(alphas_cumprod.size())

torch.Size([1000])
torch.Size([1000])
torch.Size([1000])
torch.Size([1000, 1, 1, 1])


In [16]:
i = 1

In [17]:
chunk = torch.randn((B, 1, *x.shape[-3:]), device=device)
chunk = torch.clamp(chunk, -noise_abs_max, +noise_abs_max)
print(chunk.size())
print(x.size())
x = torch.cat([x, chunk], dim=1)
print(x.size())
start_frame = max(0, i + 1 - model.max_frames)
print(start_frame)

torch.Size([1, 1, 16, 18, 32])
torch.Size([1, 8, 16, 18, 32])
torch.Size([1, 9, 16, 18, 32])
0


In [19]:
for noise_idx in reversed(range(1, ddim_noise_steps + 1)):
    ctx_noise_idx = min(noise_idx, ctx_max_noise_idx)
    print(f"min({noise_idx}, {ctx_max_noise_idx}) = {ctx_noise_idx}")
    print("ctx_noise_idx", ctx_noise_idx)
    
    t_ctx = torch.full(
                (B, i), noise_range[ctx_noise_idx], dtype=torch.long, device=device
            )
    print("t_ctx", t_ctx.size(),t_ctx)
    t = torch.full(
                (B, 1), noise_range[noise_idx], dtype=torch.long, device=device
            )
    print("t", t.size(),t)
    t_next = torch.full(
                (B, 1), noise_range[noise_idx - 1], dtype=torch.long, device=device
            )
    print(t_next.size(),t_next)
    
    t_next = torch.where(t_next < 0, t, t_next)
    t = torch.cat([t_ctx, t], dim=1)
    t_next = torch.cat([t_ctx, t_next], dim=1)
    print(t.size(),t)
    print(t_next.size(),t_next)

    # sliding window
    x_curr = x.clone()
    x_curr = x_curr[:, start_frame:]
    t = t[:, start_frame:]
    t_next = t_next[:, start_frame:]
    print(x_curr.size())
    print(t.size())
    print(t_next.size())
    print("t_next", t_next)

    # add some noise to the context
    ctx_noise = torch.randn_like(x_curr[:, :-1])
    ctx_noise = torch.clamp(ctx_noise, -noise_abs_max, +noise_abs_max)
    print(ctx_noise.size())
    noise = alphas_cumprod[t[:, :-1]].sqrt() * x_curr[:, :-1] + (1 - alphas_cumprod[t[:, :-1]]).sqrt() * ctx_noise
    print(noise.size())
    x_curr[:, :-1] = noise

    # Use DiT model to predict and remove noise
    with torch.no_grad():
        with autocast("cuda", dtype=torch.half):
            v = model(x_curr, t)
    print(v.size())

    x_start = (
                alphas_cumprod[t].sqrt() * x_curr - (1 - alphas_cumprod[t]).sqrt() * v
            )
    x_noise = ((1 / alphas_cumprod[t]).sqrt() * x_curr - x_start) / (
        1 / alphas_cumprod[t] - 1
    ).sqrt()

    # get frame prediction
    x_pred = (
        alphas_cumprod[t_next].sqrt() * x_start
        + x_noise * (1 - alphas_cumprod[t_next]).sqrt()
    )
    # Replace the generated frame in the input tensor
    print(x.size())
    x[:, -1:] = x_pred[:, -1:]
    print(x.size())
    print()
    print()

    
    if noise_idx == 3:
        break

min(100, 30) = 30
ctx_noise_idx 30
t_ctx torch.Size([1, 1]) tensor([[299]], device='cuda:0')
t torch.Size([1, 1]) tensor([[999]], device='cuda:0')
torch.Size([1, 1]) tensor([[989]], device='cuda:0')
torch.Size([1, 2]) tensor([[299, 999]], device='cuda:0')
torch.Size([1, 2]) tensor([[299, 989]], device='cuda:0')
torch.Size([1, 9, 16, 18, 32])
torch.Size([1, 2])
torch.Size([1, 2])
t_next tensor([[299, 989]], device='cuda:0')
torch.Size([1, 8, 16, 18, 32])
torch.Size([1, 8, 16, 18, 32])


EinopsError:  Error while processing rearrange-reduction pattern "(b t) d -> b t d".
 Input tensor shape: torch.Size([2, 1024]). Additional info: {'t': 9}.
 Shape mismatch, can't divide axis of length 2 in chunks of 9